<a href="https://colab.research.google.com/github/Ali-mohammadi-design/6220-Project-winter-2022/blob/main/Prompt_engineering_Lang_Chain_EBD_TRIZ_Version_19_Airplane_cabin_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install wikipedia
!pip install tiktoken
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.text_splitter import CharacterTextSplitter

In [3]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [4]:
llm= ChatOpenAI(openai_api_key=openAI_API_Key,temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


**Step1: Asking Questions**

In [5]:
json_schema={'title':'questions related to the verbs',
             'description':'questions about the verbs of the sentences',
             'type':'object',
             'properties':{
                 'question with the why': {'title':'question with the why','description':'ask why from verb of the sentence','data_type':'string'},
                 'question with the where': {'title':'question with the where','description':'ask where from verb of the sentence','data_type':'string'},
                 'question with the when': {'title':'question with the when','description':'ask when from verb of the sentence','data_type':'string'},
                 'question with the how': {'title':'question with the how','description':'ask how from verb of the sentence','data_type':'string'},
                 'question with the who': {'title':'question with the who','description':'ask who from verb of the sentence','data_type':'string'}

             },

             'required':['question with the why','question with the where','question with the when','question with the how']
             }


In [9]:
def question_asking(sentence):
  system_template1=" You are an AI assistant that gets a text and returns all nouns, pronouns, and noun phrases. Please recognize verbs but do not return verbs or propositions"
  system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
  human_template1='please list all the pronouns, nouns, and noun phrases  in the following text: \n {text}'
  human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
  prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
  chain1=LLMChain(llm=llm, prompt=prompt1, output_key='nouns')

  human_template2='Consider the following list of nouns,pronouns,and nouns phrase. please specify which ones are human and which ones are non-human.This is the list: {nouns}'
  prompt2=ChatPromptTemplate.from_template(human_template2)
  chain2=LLMChain(llm=llm, prompt=prompt2, output_key='type')

  system_template3=''' you are an AI assistant that get nouns,pronouns,and nouns phrases and  their type and ask question with what or who.
  for the human nouns,pronouns,and nouns-phrases you ask: who is + (nouns,pronouns,and nouns phrases)? \n For non-human ones, you ask: what is + (nouns,pronouns,and nouns phrases)?'''
  system_message_prompt3= SystemMessagePromptTemplate.from_template(system_template3)
  human_template3='''please consider the folowing nouns. for each noun you must ask questions with what or who depends on their type.
  for example if the nouns are teacher and sandwich you shoud return who is teacher? and what is sandwich? The nouns are : \n {type}'''
  human_message_prompt3=HumanMessagePromptTemplate.from_template(human_template3)
  noun1='Bob, school, sandwich'
  example_one= HumanMessagePromptTemplate.from_template(noun1)
  question1='who is Bob? \n what is school? \n what is sandwich?'
  example_one_output= AIMessagePromptTemplate.from_template(question1)
  prompt3=ChatPromptTemplate.from_messages([system_message_prompt3,human_message_prompt3])
  chain3=LLMChain(llm=llm, prompt=prompt3, output_key='questions')

  text=sentence
  seq_chain=SequentialChain(chains=[chain1,chain2,chain3], input_variables=['text'],output_variables=['nouns','type','questions'],verbose=False)
  result1=seq_chain(text)
  noun_questions=result1


######################################################

  system_template_verb_0=''' "You are an AI assistant who aims to get a text and separate its sentences.
  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template_verb_0)
  human_template_verb_0='''' "I'll provide a text for you. Please examine the given text, first identifying its verb phrases.
  After that, take note of each verb phrase you've identified in the provided text and return separate sentences for each verb phrase that include subject + verb phrase + object.
  \n The sentence is: \n {text}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template_verb_0)
  prompt_verb_0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  chain_verb_0=LLMChain(llm=llm, prompt=prompt_verb_0, output_key='all_sentences')


  system_template_verb_2=''' You are an AI assistant who aims to help designers to ask question.
  '''
  system_message_prompt_verb_2= SystemMessagePromptTemplate.from_template(system_template_verb_2)
  human_template_verb_2=''''
  I will provide you with a text containing subjects, verbs, objects and the sentences. Your task is to ask questions.
  generate five questions for each sentence using 'where, why, how, when, who.
  You must follow this structure:'wh question' + 'auxiliary verb' + 'subject' + 'verb' + 'object' + '?'.
  This means that for each sentence, you will create one question for each of the five question words.
  \n\nThe text to be used for this exercise is:\n\n {all_sentences}"
  '''
  human_message_prompt_verb_2=HumanMessagePromptTemplate.from_template(human_template_verb_2)
  prompt_verb_2=ChatPromptTemplate.from_messages([system_message_prompt_verb_2,human_message_prompt_verb_2])
  chain_verb_2=LLMChain(llm=llm, prompt=prompt_verb_2, output_key='all_questions')

  seq_chain2=SequentialChain(chains=[chain_verb_0,chain_verb_2], input_variables=['text'],output_variables=['all_sentences','all_questions'],verbose=False)
  result2=seq_chain2(text)
  verb_questions=result2

  template_pars_out= "Please return only the questions from the following passage. \n\n this is the text:\n\n {undesired_output}"
  chat_prompt_pars_out_template=ChatPromptTemplate.from_template(template_pars_out)
  chat_prompt=ChatPromptTemplate.from_messages([chat_prompt_pars_out_template])
  request_parsout=chat_prompt.format_prompt(undesired_output=verb_questions['all_questions']).to_messages()
  result_pars_out_final=llm(request_parsout)


################################################################

  system_template_verb_meaning_0=" You are an AI assistant that gets text. You get some text. first recoginize all the sentences in the text. Then for each sentence return the verb"
  system_message_prompt_verb_meaning_0= SystemMessagePromptTemplate.from_template(system_template_verb_meaning_0)
  human_template_verb_meaning_0='please first consider the sentences in this text and then list the vebr in each sentence: \n {all_sentences}'
  human_message_prompt_verb_meaning_0=HumanMessagePromptTemplate.from_template(human_template_verb_meaning_0)
  prompt_verb_meaning_0=ChatPromptTemplate.from_messages([system_message_prompt_verb_meaning_0,human_message_prompt_verb_meaning_0])
  chain_verb_meaning_0=LLMChain(llm=llm, prompt=prompt_verb_meaning_0, output_key='verb_list')

  system_template_verb_meaning_1=" you are an AI assistant that get verbs and ask question with what.you ask: what do you mean by + the verb?"
  system_message_prompt_verb_meaning_1= SystemMessagePromptTemplate.from_template(system_template_verb_meaning_1)
  human_template_verb_meaning_1='''please consider the folowing verbs. for each verb you must ask questions with what.
  for example, if the verb is eat you should return: what do you mean by eating? \n\n list of verbs are: {verb_list}'''
  human_message_prompt_verb_meaning_1=HumanMessagePromptTemplate.from_template(human_template_verb_meaning_1)
  prompt_verb_meaning_1=ChatPromptTemplate.from_messages([system_message_prompt_verb_meaning_1,human_message_prompt_verb_meaning_1])
  chain_verb_meaning_1=LLMChain(llm=llm, prompt=prompt_verb_meaning_1, output_key='verb_meaning_questions')

  text=sentence
  seq_chain3=SequentialChain(chains=[chain_verb_meaning_0,chain_verb_meaning_1], input_variables=['all_sentences'],output_variables=['verb_list','verb_meaning_questions'],verbose=False)
  result3=seq_chain3(text)
  verbs_meaning=result3


  #####################################################################################

  system_template0=''' You are an AI assistant who aims to help designers to extract as much information as possible from a design problem.
  You get a design problem.
  You should ask six questions with the following format:
  1- what do you mean by + the whole design problem +?
  2- Where do you need + the whole design problem + ?
  3- when do you need + the whole design problem + ?
  4- why do you need + the whole design problem + ?
  5- How do you need + the whole design problem + ?
  6- who is responsible for + the whole design problem + ?

  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
  human_template0=''''
  \n The design problem is: \n {text}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
  prompt0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  request=prompt0.format_prompt(text=text).to_messages()
  result0=llm(request)
  verb_questions2=result0

  question_final_list= f"\n \n {noun_questions['questions']}   \n\n{verbs_meaning['verb_meaning_questions']}  \n \n{ result_pars_out_final.content}\n\n {verb_questions2.content} "
###############################################################################

  system_template0=''' You are an AI assistant who can recognize repetetive concepts.
  You will receive a list of questions, some of these questions may be repetitive or asking about similar concept.
  Your objective is to provide a list of unique questions without modifying the wording of the original questions or making any other changes to them.

  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
  human_template0=''''
  \n This is the list of questions: \n {question_final_list}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
  prompt0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  request=prompt0.format_prompt(question_final_list=question_final_list).to_messages()
  result0=llm(request)
  question_final_list_unique=result0

  return print(f"these are the questions should be answered:\n\n{question_final_list_unique.content} \n\n\n...........\n\n The preliminary questions:\n\n{question_final_list}")


In [10]:
question_asking('Design a passenger cabin that includes maximum number of seats while ensuring passenger comfort.')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


these are the questions should be answered:

1. Who is passenger?
2. What is cabin?
3. What is number?
4. What is seats?
5. What is comfort?
6. What is passenger cabin?
7. What is maximum number of seats?
8. What is passenger comfort?

1. What do you mean by designing?
2. What do you mean by including?
3. What do you mean by ensuring?

1. Where do you design a passenger cabin?
2. Why do you design a passenger cabin?
3. How do you design a passenger cabin?
4. When do you design a passenger cabin?
5. Who designs a passenger cabin?

1. Where does the passenger cabin include the maximum number of seats?
2. Why does the passenger cabin include the maximum number of seats?
3. How does the passenger cabin include the maximum number of seats?
4. When does the passenger cabin include the maximum number of seats?
5. Who includes the maximum number of seats in the passenger cabin?

1. Where do you ensure passenger comfort?
2. Why do you ensure passenger comfort?
3. How do you ensure passenger com

**Second Stage: Answering the questions**

In [11]:
class object_lifecycle():
  def __init__(self,First_stage,Second_stage,Third_stage,Fourth_stage):
     self.first_stage =First_stage
     self.Second_stage=Second_stage
     self.Third_stage=Third_stage
     self.Fourth_stage=Fourth_stage

In [12]:
json_schema={'title':'scientist',
             'description':'four stages of the creation of the object',
             'type':'object',
             'properties':{
                 'first_stage': {'title':'first_stage','description':'The First stage of the lifecycle of this object','data_type':'string'},
                 'second_stage': {'title':'Second_stage','description':'The second stage of the lifecycle of this object','data_type':'string'},
                 'third_stage': {'title':'third_stage','description':'The third stage of the lifecycle of this object','data_type':'string'},
                 'last_stage': {'title':'last_stage','description':'The last stage of the lifecycle of this object','data_type':'string'},
             },

             'required':['first_stage','second_stage','thrid stage','last stage']
             }

In [13]:
model= ChatOpenAI(openai_api_key=openAI_API_Key, temperature=0)

In [14]:
def What_OR_Who_is(name):

  system_template=" you are a dictionary that could help designers to define {objects}. You would define {objects} clearly and briefly in one sentence"
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= ' What is {objects}?'
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  request=chat_prompt.format_prompt(objects=name).to_messages()
  definition=llm(request)



  system_template=" you are an AI assistant that could help designers to understand the lifecycle of {objects}. You would explain the lifecycle of {objects} clearly and very breifly in just four stages and provide a  short explanation for every stage "
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= 'I would define lifecycle as events that something passes to reach the point that it is. For instance the lifecycle of an engineer is studying in school, getting good marks, going to colledge, going to university, finding an engineer job. What is life cycle of {objects}'
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  chain=create_structured_output_chain(json_schema,llm, chat_prompt,verbose=False)
  result=chain.run(objects=name)
  obj=object_lifecycle(result['first_stage'],result['second_stage'],result['third_stage'],result['last_stage'])


  system_template=''' Every {objects} is related to it's environment.
  we would have three types of environment: 1- built environment, 2-human environment 3- nature environment.
  Built environment: Every thing directly related to the {objects} and made by human.
  Human environment: Every human being directly related to the {objects}.
  Nature environment: Every natural things like weather, water etc that is directly connect to {objects}.
  When we wnat to describe the environment of {objects} we have to show the relation between {objects} and the environment in senctnces.
  you are an AI assistant that could help designers to understand the environment of {objects}.
  You would explain the environment of {objects} with single sentences.You would prefer to use action verbs in the sentences'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= '''{lifecycle_stage} is one of the stages of lifecycle of {objects}.
   Consider {objects} in this stage and determine every part of the environment of {objects} in the separate sentences'''
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  model_request= chat_prompt.format_prompt(lifecycle_stage=obj.first_stage, objects=name).to_messages()
  env_in_first_stage=model(model_request)

  model_request2= chat_prompt.format_prompt(lifecycle_stage=obj.Second_stage, objects=name).to_messages()
  env_in_second_stage=model(model_request2)

  model_request3= chat_prompt.format_prompt(lifecycle_stage=obj.Third_stage, objects=name).to_messages()
  env_in_third_stage=model(model_request3)

  model_request4= chat_prompt.format_prompt(lifecycle_stage=obj.Fourth_stage, objects=name).to_messages()
  env_in_fourth_stage=model(model_request4)


  result=f'''
  {name} definition: {definition.content}
  \n {name} usually has a specific life cycle and some environment in its lifecycle.
  \n The first stage of the life cycle of {name} is {obj.first_stage}. In this stage, the environment is classified as the following:\n{env_in_first_stage.content}
  \n The second stage of the life cycle of {name} is {obj.Second_stage}. In this stage, the environment is classified as the following:\n{env_in_second_stage.content}
  \n The third stage of the life cycle of {name} is {obj.Third_stage}. In this stage, the environment is classified as the following:\n{env_in_third_stage.content}
  \n The fourth stage of the life cycle of {name} is {obj.Fourth_stage}. In this stage, the environment is classified as the following:\n{env_in_fourth_stage.content}
  '''
  return print(result)

In [20]:
What_OR_Who_is('design')


  design definition: Design is the intentional and thoughtful creation of solutions that address a specific problem or need, while also considering aesthetics, functionality, and user experience.
  
 design usually has a specific life cycle and some environment in its lifecycle.
  
 The first stage of the life cycle of design is Research. In this stage, the environment is classified as the following:
In the built environment of design research, designers gather information, analyze data, and explore potential solutions. 
In the human environment of design research, designers collaborate with stakeholders, conduct interviews, and gather feedback to understand user needs. 
In the nature environment of design research, designers consider external factors such as market trends, technological advancements, and cultural influences.
  
 The second stage of the life cycle of design is Design. In this stage, the environment is classified as the following:
Built environment: The design process 

**Third_stage: Identifying action verbs**

In [ ]:
def active_verb_extraction(text):
    system_template1='''An action verb, also known as a dynamic verb, is a type of verb that describes an action or something that a person or thing does (e.g., “I  run”).
    Action verbs differ from stative verbs, which describe a state of being (e.g., “believe,” “want”, "is", "are", "am", "have", "has", "become", "feel").
    Note: Gerund (verb +ing) are not action verb. While run is an action verb, runing is not an action verb.
    If there is no action verb in the sentence, you should return "there is no action verb"
    As an AI research assistant who can analyze sentences, your task is to identify action verbs in a sentence'''
    system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
    human_template1='Please return the action verbs in this text:\n {text}'
    human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
    prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
    chain1=LLMChain(llm=llm, prompt=prompt1, output_key='action_verbs')


    system_template2= '''As an AI assistant,You would get some action verbs.
    Your task is to identify these action verbs in the text and split the text to independent sentences.
    Each sentence has only one active verb.
    If there is no action verb you do not need to generate sentence. Just mention, this sentence does not have any action verb.
    Your output is only a sentence without extra explaination.
    '''
    system_message_prompt2= SystemMessagePromptTemplate.from_template(system_template2)
    human_template2='''Regard these action verbs:\n {action_verbs}.
    In the following text, identify these action verbs and return the action verb with its subject and object in the form of a sentence.
    This is the text:\n {text}'''
    human_message_prompt2=HumanMessagePromptTemplate.from_template(human_template2)
    prompt2=ChatPromptTemplate.from_messages([system_message_prompt2,human_message_prompt2])
    chain2=LLMChain(llm=llm, prompt=prompt2, output_key='active_sentence')

    seq_chain=SequentialChain(chains=[chain1,chain2], input_variables=['text'],output_variables=['action_verbs','active_sentence'],verbose=False)

    sentences = text.split(".")
    active_sentences_text=''

    for l in sentences:
       out_put=seq_chain(l)
       active_sentences_text=active_sentences_text + f"{out_put['active_sentence']}\n"

    print(f"the first stage is: \n {active_sentences_text} \n END \n")


    system_template='''
   An action verb, also known as a dynamic verb, is a type of verb that describes an action or something that a person or thing does (e.g., “I  run”).
    Action verbs differ from stative verbs, which describe a state of being (e.g., “believe,” “want”, "have", "has", "become", "feel").
    Note:verb +ing is not action verb. While run is an action verb, runing is not an action verb.
    You are an AI assitant who is expert in recognizing sentences with action verbs.
    I would provide you some sentences. Some of these sentences have one or more action verbs and some of them do not have any action verbs.
    Please only retrun the sentences with action verbs.
  '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= 'Please consider the following sentences and only return sentences that have one or more action verbs: \n {active_sentences_text} '
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])
    request=prompt0.format_prompt(active_sentences_text=active_sentences_text).to_messages()
    result=llm(request)
    return print(result.content)





In [ ]:
text='''
Long distance swimming training is a structured program designed to improve endurance, stamina, and technique for swimming distances typically ranging from 1,500 meters to several kilometers.
The first stage of the life cycle of long distance swimming training is Setting goals.
The swimmer first should design a plan with specific milestones and time lines.
coaches play a crucial role in guiding swimmers, providing motivation, and monitoring progress towards their goals.
water temperature, currents, and weather conditions must be taken into account when planning and executing training sessions.

The second stage of the life cycle of long distance swimming training is Building endurance and strength through consistent training.

The Swimmers should use a structured training plan with designated swimming lanes and timing systems to track progress.

The swimmers should Collaborate with coaches, trainers, and fellow swimmers to provide support and motivation during intense training sessions.

The swimmers should Utilize natural bodies of water such as lakes or oceans to simulate race conditions and adapt to varying water temperatures and currents.

The third stage of the life cycle of long distance swimming training is Fine-tuning technique and focusing on mental preparation.

Athletes fine-tune their technique by practicing strokes in controlled pool settings.

coaches play a crucial role in guiding athletes through mental preparation techniques to enhance performance.

swimmers adapt to different water conditions such as temperature and currents to build resilience and endurance.

We would design a pool in the university

Swimmers cannot always use oceans and lakes to swim. Thus we need a pool which is usable and available always.

The pools have the potential to be installed and utilized in a variety of locations. However, it is necessary to have a dedicated space to accommodate them.

The swimmer could be use it whenever he wants.

'''

In [ ]:
active_verb_extraction(text)

the first stage is: 
 - Swimming training is designed to improve endurance, stamina, and technique. 
- Distances typically ranging from 1,500 meters to several kilometers.
Setting goals is the action in the sentence.
The swimmer should design a plan.
Coaches play a crucial role in guiding swimmers.  
Coaches provide motivation.  
Coaches monitor progress towards their goals.
Water temperature, currents, and weather conditions must be taken into account when planning training sessions.  
Water temperature, currents, and weather conditions must be taken into account when executing training sessions.
Building endurance and strength through consistent training.
The Swimmers should use a structured training plan.  
Swimmers should use designated swimming lanes.  
Swimmers should use timing systems to track progress.
The swimmers should collaborate with coaches, trainers, and fellow swimmers.  
They provide support and motivation during intense training sessions.
The swimmers should utilize 

**Fourth Stage: Establishing Functional analysis**

In [ ]:
json_schema2={'title':'Functional analysis',
             'description':'Identfying tools, objcets and their interactions',
             'type':'object',
             'properties':{
                 'Tool': {'title':'tool','description':'The objcet that change another object or protect another object to be changed','data_type':'string'},
                 'Object': {'title':'Object','description':'The Object which is changed or being protected form a change','data_type':'string'},
                 'How tool affect the object': {'title':'How tool affect on the object','description':'How tool affect on the object','data_type':'string'},
                 'How object affect the tool': {'title':'How object affect the tool','description':'How object affect the tool','data_type':'string'},
             },

             'required':['Tool','Object','How tool affect the object','How object affect the tool']
             }

Note: .split() would convert a text to a list. If you determine a sign like "." or "," in the (), like: text.split("."), the function would split the text ib every "." in generate a list of the components

In [ ]:
def functional_analysis(text):
  system_template='''you are an AI assistant that could analyze a sentence and specify tool, object and the impact of tools on object and the impact of object on tool'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)

  text1= 'Alex write a sentence'
  example_one= HumanMessagePromptTemplate.from_template(text1)
  text1_out_put= 'Tool: Alex, Object: Sentence, How tool affect the object: creating the sentence, How object affect the tool:None'
  example_one_output= AIMessagePromptTemplate.from_template(text1_out_put)

  text2='Ali eats a sandwich'
  example_two= HumanMessagePromptTemplate.from_template(text2)
  text2_out_put= 'Tool: Ali, Object: Sandwich,  How tool affect the object: consuming the sandwich, How object affect the tool:None'
  example_two_output= AIMessagePromptTemplate.from_template(text2_out_put)

  text3='Dog walks near the school'
  example_three= HumanMessagePromptTemplate.from_template(text3)
  text3_out_put= 'specified action means tool does something that impact the object. There is imapct on dog or school. Thus there is no tool, and object'
  example_three_output= AIMessagePromptTemplate.from_template(text3_out_put)

  text4='The manager interview a job applicant'
  example_four= HumanMessagePromptTemplate.from_template(text4)
  text4_out_put= 'Tool: Job applicant, Object: Manger,  How tool affect the object: increasing the information of manager, How object affect the tool:None '
  example_four_output= AIMessagePromptTemplate.from_template(text4_out_put)

  Human_prompt= '''In every action there is a tool and object. Action means tool does something that causes object to change or to be maintained from the change.
  Thus in every action there are two material object (one is tool and the other is object) that are interacting with each other.
  In This interaction the tool must impact the object. If there is no impact on the object, we can conclude that there is no action.
  The object that impact on the other one is called as a tool and the one that is got affected is the bject.
  Please consider these sentences:\n {text} \n please clarify waht are tools and objects in every sentence.
  Please also clarify how the tool affects the object and how the object affect the tool'''
  prompt= HumanMessagePromptTemplate.from_template(Human_prompt)
  chat_prompt=ChatPromptTemplate.from_messages([prompt,system_message_prompt])
  chain=create_structured_output_chain(json_schema2,llm, chat_prompt,verbose=False)
  sentences = text.split(".")
  result=[]
  for l in sentences:
   out_put=chain.run(text=l)
   print(f"\n the sentence is: \n {l}")
   print(f" the functional analysis is: \n {out_put}")
   result.append(out_put)

In [ ]:
text='''
Active sentences:
I finished high school.
I entered the university there.
I went to military service.
I worked for a year.
I accepted as a student in Canada.
I came to Concordia university.
I do a PhD program.
I am working for a professor at Concordia University.
I go to some cultural centers.
I voluntarily work with them.
I am using social applications.
I got injured on my back.
I must exercise to relieve my backache.
I would work with Python.
I have a part-time job.
I meet a few people.
I have to finish my degree.
I find a job.
You have to understand it.
You have to talk to her.
You can use romantic restaurants.
She can spend quality time with her partner in beautiful parks.
You have to use compliments.
The girl is influenced by her friends and family members.
You need to buy some gifts.
You have to date with her more and more.
She gets more committed to you.
To find new places, ideas, or possibilities through curiosity and adventurousness.
We need some friends.
We ask humans.
We can also ask other humans in the specific environment about a subject.
After discovering, we have to evaluate the thing.
We can use others' experience to evaluate the founded object.
You can use search engines.
You can use maps.
You can investigate something.
I can find a girlfriend.
The girls go.
Girls go to university.
Girls go to parties.
Girls go to the park.
Girls go to the museum.
Girls go to offices.
Girls work.
They would go to classes.
They would attend workshops.
They would go to Gyms.
They would go to cafes.
They would go to shopping centers.
They use online platforms.
They would go to travel.
They would go to libraries.
I go to where the girls are.

'''

In [ ]:
functional_analysis(text)


 the sentence is: 
 
Active sentences:
I finished high school
 the functional analysis is: 
 {'Tool': 'I', 'Object': 'high school', 'How tool affect the object': 'Finishing high school is the action performed by the tool (I), resulting in the object (high school) being completed or achieved.', 'How object affect the tool': 'The completion of high school may lead to personal growth, new opportunities, or further education for the tool (I).'}

 the sentence is: 
 
I entered the university there
 the functional analysis is: 
 {'Tool': 'I', 'Object': 'university', 'How tool affect the object': 'Entering the university', 'How object affect the tool': 'Being the destination of the action'}

 the sentence is: 
 
I went to military service
 the functional analysis is: 
 {'Tool': 'I', 'Object': 'military service', 'How tool affect the object': 'I went to', 'How object affect the tool': 'I went to'}

 the sentence is: 
 
I worked for a year
 the functional analysis is: 
 {'Tool': 'I', 'Object':

**Fifth Stage: Idea Generation with TRIZ tools**

In [ ]:
llm_h= ChatOpenAI(openai_api_key=openAI_API_Key,temperature=1)

In [ ]:
def TRIZ_principles(problem, contradiction, principles):
  system_template='''you are an AI assistant that could get a design problem and its contradiction and generate ideas regarding TRIZ principles.
  The out-put text should clearly state generated ideas and provide an example for each idea to better explain them.
  Your out-put is only the ideas and examples witout extra explaination.'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)

  Human_prompt= '''TRIZ is an acronym for "Theory of Inventive Problem Solving."
   It is a problem-solving methodology that originated in Russia and was developed by Genrich Altshuller and his colleagues in the mid-20th century.
   The 40 Principles of TRIZ are a set of guidelines formulated by Genrich Altshuller, the founder of TRIZ.
   These principles are based on his analysis of thousands of patents across various industries.
   They serve as strategies for overcoming contradictions and generating inventive solutions for problem-solving and innovation.
   According to TRIZ in every design problem there is a contradiction that would make this design problme.
   Now, I would like to give you a design problem with its contradiction. Please resolve it using Principle {number} of TRIZ.
   \n Here's the problem:\n{problem}
   \n This is the contradiction recognized in the problem:\n {contradiction}.
   '''
  prompt= HumanMessagePromptTemplate.from_template(Human_prompt)
  chat_prompt=ChatPromptTemplate.from_messages([prompt,system_message_prompt])
  ideas=''
  Principle_number=principles
  for x in range(Principle_number):
    model_request= chat_prompt.format_prompt(number= str(x+1),problem=problem, contradiction=contradiction).to_messages()
    result=llm_h(model_request)
    print(f"\n{result.content}")
    ideas= ideas + result.content


    text_splitter=CharacterTextSplitter.from_tiktoken_encoder(chunk_size=600)
    splitted_ideas=text_splitter.split_text(ideas)



    system_template='''
   You are an AI assistant who is an expert in removing repetitive ideas.
   I will provide you with a list of ideas and their examples.
   Some of these ideas may be repetitive, so please remove any duplicates and return a list of unique ideas with their examples.
  '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= 'Please consider the list of ideas and only return a list of unique ideas. \n list of ideas: \n {ideas} '
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    summerized_ideas= ''

    for k in splitted_ideas:
     request=prompt0.format_prompt(ideas=k).to_messages()
     summerized_ideas = summerized_ideas + llm(request).content



    system_template='''
   You are an AI assistant who is an expert in removing repetitive ideas.
   I will provide you with a list of ideas and their examples.
   Some of these ideas may have repetitive concept, so please remove ideas with the same concepts and return a list of unique ideas with their examples.
   '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= 'Please consider the list of ideas and only return a list of unique ideas. \n list of ideas: \n {summerized_ideas} '
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    request2=prompt0.format_prompt(summerized_ideas=summerized_ideas).to_messages()
    second_summerized_ideas=llm(request2).content




    system_template='''
   You are an AI assistant who is an expert to detremine which experts are required in a design team for resolving a design problem.
   you get the design problem, contradiction in the design problem and generated ideas.
   You would return the experts needed to validate and implement the ideas to resolve the design problem and create a novel product.
   '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= '''this is the design problem: \n {problem}
    \n\n this is the contradiction inside the design problem: {contradiction}
    \n\n this is the generated ideas:{final_ideas}
    '''
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    request3=prompt0.format_prompt(problem=problem, contradiction=contradiction, final_ideas=second_summerized_ideas).to_messages()
    design_team=llm(request3).content






  return print(f" \n \n This is the contradiction that we face:\n{contradiction}.\n \n \n Regarding TRIZ principles from number 1 to number {Principle_number}, we would have these ideas:\n\n{summerized_ideas} \n\n\n--------------\n\n\n second_summerized ideas:\n{second_summerized_ideas} \n\n\n These experts can help you: \n{design_team}")







In [ ]:
problem='''Design a pool suitable for uninterrupted long distance swimming'''
contradiction='''A swimming line has an end, But We must not have an end because it would destroy the rhythm of the swimmer for uninterrupted long-distance swimming'''

In [ ]:
TRIZ_principles(problem=problem, contradiction=contradiction, principles=15)


Idea 1: Principle 1 - Segmentation
Example: Create a pool with a moving belt mechanism at the bottom that continuously moves in the opposite direction of the swimmer, allowing them to swim in one direction indefinitely without reaching an end.

Idea 2: Principle 1 - Segmentation
Example: Design a pool with a circular loop layout, where swimmers can continuously swim in a circular path without reaching an end, providing uninterrupted long-distance swimming.

Idea 3: Principle 1 - Segmentation
Example: Implement a pool design with a series of interconnected lanes that loop back on themselves, allowing swimmers to swim back and forth without hitting a wall or turning point, enabling uninterrupted long-distance swimming.

Principle 2: Taking Out: Remove a component that causes harm or contradiction without losing the essential function.

Idea 1: Remove the need for a physical swimming line.
Example: Implement an underwater guidance system that uses sensors to create a virtual line for swi